## Import Packages
Import common packages

In [1]:
import os
import datetime
import numpy as np

Tell python where your local copy of *chmap* is located

In [2]:
import sys
sys.path.append("/Users/turtle/GitReps/CHD")

Import chmap functions

In [3]:
import chmap.utilities.datatypes.datatypes as psi_dtypes
import chmap.database.db_funs as db_funcs
import chmap.database.db_classes as db_class
import chmap.data.corrections.iit.IIT_pipeline_funcs as iit_funcs
import chmap.data.corrections.lbcc.LBCC_theoretic_funcs as lbcc_funcs

## Set parameters

In [4]:
####### ------ UPDATABLE PARAMETERS ------ #########
# TIME RANGE FOR LBC CORRECTION AND IIT HISTOGRAM CREATION
lbc_query_time_min = datetime.datetime(2011, 2, 1, 0, 0, 0)
lbc_query_time_max = datetime.datetime(2011, 2, 2, 0, 0, 0)

# TIME RANGE FOR FIT PARAMETER CALCULATION
calc_query_time_min = datetime.datetime(2011, 4, 1, 0, 0, 0)
calc_query_time_max = datetime.datetime(2011, 10, 1, 0, 0, 0)
weekday = 0  # start at 0 for Monday
number_of_days = 180  # days for moving average
# TIME WINDOWS FOR IMAGE INCLUSION
image_freq = 2      # number of hours between window centers
image_del = np.timedelta64(30, 'm') # one-half window width

# TIME RANGE FOR IIT CORRECTION AND IMAGE PLOTTING
iit_query_time_min = datetime.datetime(2011, 4, 1, 0, 0, 0)
iit_query_time_max = datetime.datetime(2011, 10, 1, 0, 0, 0)
plot = True  # true if you want to plot resulting images
n_images_plot = 1  # number of images to plot

# TIME RANGE FOR HISTOGRAM CREATION
hist_query_time_min = datetime.datetime(2011, 4, 1, 0, 0, 0)
hist_query_time_max = datetime.datetime(2011, 10, 1, 0, 0, 0)

# INSTRUMENTS
inst_list = ["AIA", "EUVI-A", "EUVI-B"]
ref_inst = "AIA"  # reference instrument to fit histograms to
wavelengths = [193, 195]

# declare map and histogram-binning parameters
n_mu_bins = 18
n_intensity_bins = 200
R0 = 1.01
log10 = True
lat_band = [-np.pi / 2.4, np.pi / 2.4]

# DATABASE FILE-SYSTEM PATHS
raw_data_dir = "/Volumes/extdata2/CHMAP_DB_example/raw_images"
hdf_data_dir = "/Volumes/extdata2/CHMAP_DB_example/processed_images"

# DATABASE CONNECTION
create = True  # set to False to disallow writes to the database
db_type = "sqlite"
sqlite_path = "/Volumes/extdata2/CHMAP_DB_example/chmap_example.db"

In [6]:
# connect to database
db_session = db_funcs.init_db_conn(db_type, db_class.Base, db_loc=sqlite_path)

Attempting to connect to SQLite DB server /Volumes/extdata2/CHMAP_DB_example/chmap_example.db
Connection successful



## Choose an image for demonstration

In [18]:
query_instrument = ["EUVI-A", ]
query_pd_all = db_funcs.query_euv_images(db_session=db_session, time_min=lbc_query_time_min,
                                         time_max=lbc_query_time_max, instrument=query_instrument,
                                         wavelength=wavelengths)
use_image = query_pd_all.loc[4]
print(use_image)

data_id                                                    30763
date_obs                                     2011-02-01 10:00:30
instrument                                                EUVI-A
wavelength                                                   195
distance                                             143867000.0
cr_lon                                                    288.31
cr_lat                                                  -4.39192
cr_rot                                                    2106.2
flag                                                           0
time_of_download                             2020-09-30 18:54:44
fname_raw           2011/02/01/sta_euvi_20110201T100030_195.fits
fname_hdf           2011/02/01/euvia_lvl2_20110201T100030_195.h5
Name: 4, dtype: object


## Apply limb correction

In [19]:
# apply limb correction to chosen image
original_los, lbcc_image, mu_indices, use_indices, theoretic_query = lbcc_funcs.apply_lbc_2(
    db_session, hdf_data_dir, use_image, n_intensity_bins=n_intensity_bins, R0=R0)

## Generate equatorial histogram

In [23]:
# calculate IIT histogram from limb-corrected image
temp_hist = psi_dtypes.LBCCImage.iit_hist(lbcc_image, lat_band, log10)

# query IIT method_id
meth_name = "IIT"
method_id = db_funcs.get_method_id(db_session, meth_name)

# create IIT histogram datatype
iit_hist = psi_dtypes.create_iit_hist(lbcc_image, method_id[1], lat_band, temp_hist) 

Add histogram to database

In [ ]:
# add IIT histogram and meta data to database
# db_funcs.add_hist(db_session, iit_hist)